---
title: Login/Signup
courses: {'compsci': {'week': 6}}
type: tangibles
layout: base
---

# Building a Secure Login/Signup System

## 1. Introduction (Vibha)

- Brief overview of the importance of secure authentication.
- Mention of the frontend and backend components.




## 2. Frontend Components Anusha

#### a. User Interface (UI)


[Login/Register/CRUD Demonstration](https://drive.google.com/file/d/1IDUZM-ztixoc51dGLEeTeEQB3StiVIT1/view?usp=sharing)

- Explanation of Login and Signup page designs.
- Showcase of UI elements and navigation.

#### b. User Interaction

- Demonstration of input validation.
- Explanation of how authentication requests are sent to the backend (drawing)

## 3. Backend Components Isabel


To create and register a user, you can using the existing code in user.py in the api folder and update it, like this:

In [ ]:
class _CRUD(Resource):  # User API operation for Create, Read.  THe Update, Delete methods need to be implemented
        def post(self): # Create method
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password and dob
            password = body.get('password')
            dob = body.get('dob')
            coins = 0
            
            
            tracking = body.get('tracking') #validate tracking
            #
            exercise = body.get('exercise') #validate exercise

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, #user name
                      uid=uid, tracking=tracking, exercise=exercise, dob=dob, coins=coins)

# The `LoginAPI` class defines a resource for handling user login in a Flask application.

class LoginAPI(Resource):
    def post(self):
        data = request.get_json()

        # Retrieve uid and password from the request data
        uid = data.get('uid')
        password = data.get('password')

# The post method handles HTTP POST requests.
# Retrieves JSON data from the request body, expecting 'uid' and 'password' fields.

In [ ]:

        # Check if uid and password are provided
        if not uid or not password:
            response = {'message': 'Invalid credentials'}
            return make_response(jsonify(response), 401)

# Input Validation:
# Checks if both 'uid' and 'password' are provided in the request.
# If not, it returns a JSON response with an 'Invalid credentials' message and a 401 (Unauthorized) status.


        # Retrieve user by uid from the database
        user = User.query.filter_by(_uid=uid).first()

# Database Query:
# Retrieves a user from the database based on the provided 'uid'.

        # Check if the user exists and the password is correct
        if user and user.is_password(password):

# User Authentication:
# Verifies if the user exists in the database and if the provided password matches the stored password (using a method is_password on the user object).


            # Perform login operations here (if needed)

            # Construct the response with the user's name included
            response = {
                'message': 'Logged in successfully',
                'user': {
                    'name': user.name,  
                    'id': user.id
                }
            }
            return make_response(jsonify(response), 200)

Successful Login Response:
If the user exists and the password is correct, it constructs a JSON response indicating a successful login, including the user's name and ID.
python. If the user does not exist or the password is incorrect, it returns a JSON response with an 'Invalid UID or password' message and a 401 status.
LogoutAPI Resource:







## 4. Login Process/Signup   Justin  Frontend

#### a. Step-by-step walkthrough

- Frontend and backend interactions during the login/signup process.
- Demonstration of how the user gains access to protected routes.










### 6. Best Practices and Additional Features Vibja

- Brief discussion on additional security features (e.g., email verification, two-factor authentication).
- Emphasis on following best practices for security and privacy.
<!--
Suggestions
Use HTTPS (SSL/TLS):
Ensure that all communication between clients and the server is encrypted using HTTPS. This helps prevent man-in-the-middle attacks and protects sensitive information during transmission.

Token-Based Authentication:
Implement token-based authentication, such as JSON Web Tokens (JWT) or OAuth, to securely manage user sessions. Tokens should be generated securely, have a limited lifespan, and be securely stored on the client side.

Secure Password Storage:
Hash and salt passwords before storing them in the database. Use strong hashing algorithms (e.g., bcrypt) to protect user passwords from being exposed in the event of a data breach.

Authentication Rate Limiting:
Implement rate limiting to prevent brute-force attacks on login endpoints. This can involve limiting the number of login attempts within a specified time period to mitigate the risk of unauthorized access.

Secure User Registration:
Implement validation and sanitization checks on user registration inputs to prevent injection attacks. Verify the authenticity of email addresses and usernames during the registration process.

Multi-Factor Authentication (MFA):
Encourage or require users to enable MFA to add an additional layer of security. This can involve using one-time codes sent via SMS, email, or authenticator apps.

Session Management:
Implement secure session management practices. Ensure that session tokens are securely stored and transmitted, and consider implementing session timeout and re-authentication mechanisms.

Cross-Site Request Forgery (CSRF) Protection:
Implement measures to protect against CSRF attacks. Use anti-CSRF tokens and ensure that requests from legitimate users originate from trusted sources.

Input Validation and Sanitization:
Validate and sanitize all user inputs to prevent injection attacks, such as SQL injection or Cross-Site Scripting (XSS). Use parameterized queries for database interactions.

Logging and Monitoring:
Implement comprehensive logging for login/signup activities. Monitor and log failed login attempts, unusual patterns, and potential security events to detect and respond to security incidents.

API Key Security:
If applicable, secure API keys used for authentication and authorization. Ensure that keys are kept confidential, rotated regularly, and that access is restricted to only necessary entities.

Regular Security Audits and Updates:
Conduct regular security audits of your codebase and dependencies. Stay updated on security best practices and promptly apply patches and updates to address any vulnerabilities.
 -->